## Get SysML v2 classes
A notebook to get a list of the SysML v2 classes and generate the vocabToShape div sections needed in sysml-shapes.html.



In [1]:
# Read and parse the sysml-vocab.ttl file
from rdflib import Graph, URIRef, Literal, Namespace, RDF

# some useful RDF namespaces
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace('http://www.w3.org/2000/01/rdf-schema#')
dcterms = Namespace('http://purl.org/dc/terms/')
oslc = Namespace('http://open-services.net/ns/core#')
oslc_am = Namespace('http://open-services.net/ns/am#')
oslc_sysmlv2 = Namespace('http://open-services.net/ns/sysmlv2#')

g = Graph()
g.parse('sysml-vocab-full.ttl')


<Graph identifier=N9b094db091f84758afb5415fcc1088b3 (<class 'rdflib.graph.Graph'>)>

In [ ]:
# get a list of the classes
classes = g.subjects(rdf.type, rdfs.Class)

# and now create the shapeToSpec div for each class
for c in classes:
    name = str(c)
    name = name[name.index('#')+1:]
    print(
f"""

<div
        title="Constraints for {name}"
        data-include="./sysml-shapes.ttl#{name}Shape"
        data-oninclude="shapeToSpec"
        data-include-sync="true"
        data-include-replace="true"
        data-include-format="html"
      ></div>
"""        
    )

In [ ]:
# Convert the rdfs:comments to a single paragraph of plain text
from bs4 import BeautifulSoup
import copy

g2 = copy.deepcopy(g)
for s,p,o in g2.triples((None, rdfs.comment, None)):
    # Get just the first paragraph of the comment
    root = BeautifulSoup(str(o))
    comment = root.find('p').get_text()  # the first paragraph with the HTML stripped out
    # remove the triple
    g.remove((s,p,o))
    # add the new comment
    g.add((s,p,Literal(comment)))

g.serialize(destination='../sysml-vocab.ttl')



In [ ]:
# Set shape dcterms:descriptions to just the first paragraph
g = Graph()
g.parse('sysml-shapes-full.ttl')

g2 = copy.deepcopy(g)
for s,p,o in g2.triples((None, dcterms.description, None)):
    # Get just the first paragraph of the comment
    root = BeautifulSoup(str(o))
    description = root.find('p')  # the first paragraph but keep the markdown
    # remove the triple
    g.remove((s,p,o))
    # add the new description
    g.add((s,p,Literal(description)))

g.serialize(destination='../sysml-shapes.ttl')



In [10]:
# Add the rdf:Property and rdfs:Class rdfs:isDefinedBy  oslc_sysmlv2: 

g = Graph()
g.parse('../sysml-vocab.ttl')

oslc_sysmlv2 = URIRef('http://open-services.net/ns/sysmlv2#')

g2 = copy.deepcopy(g)
for s,p,o in g2.triples((None, rdf.type, rdf.Property)):
    g.add((s,rdfs.isDefinedBy, oslc_sysmlv2))

for s,p,o in g2.triples((None, rdf.type, rdfs.Class)):
    g.add((s,rdfs.isDefinedBy, oslc_sysmlv2))

g.serialize(destination='../sysml-vocab.ttl')

<Graph identifier=Nedc6b4d29f3a43b2947d38ced2342ef5 (<class 'rdflib.graph.Graph'>)>